https://github.com/tensorflow/models/tree/master/inception

https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/image_retraining/retrain.py

In [1]:
import tensorflow as tf
import numpy as np
import os
import random
from show_graph import show_graph
from nbutil import imshow_multi, to_pil

In [2]:
# from https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/image_retraining/retrain.py

BOTTLENECK_TENSOR_NAME = 'pool_3/_reshape:0'
JPEG_DATA_TENSOR_NAME = 'DecodeJpeg/contents:0'
RESIZED_INPUT_TENSOR_NAME = 'ResizeBilinear:0'
SOFTMAX_TENSOR_NAME = 'softmax:0'
PATH = '/Users/nateparrott/Documents/SW/models/inception-2015-12-05'

def create_inception_graph(path):
  """"Creates a graph from saved GraphDef file and returns a Graph object.
  Returns:
    Graph holding the trained Inception network, and various tensors we'll be
    manipulating.
  """
  with tf.Session() as sess:
    model_filename = os.path.join(
        path, 'classify_image_graph_def.pb')
    with open(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        bottleneck_tensor, jpeg_data_tensor, resized_input_tensor, softmax_tensor = (
          tf.import_graph_def(graph_def, name='', return_elements=[
              BOTTLENECK_TENSOR_NAME, JPEG_DATA_TENSOR_NAME,
              RESIZED_INPUT_TENSOR_NAME, SOFTMAX_TENSOR_NAME]))
    return graph_def, bottleneck_tensor, jpeg_data_tensor, resized_input_tensor, softmax_tensor

# graph_def, bottleneck, jpeg_input, resized_image_input, softmax_tensor = create_inception_graph(PATH)


In [3]:
def create_image_queue(filenames, batch_size=16, grayscale=False):
    filename_tensor = tf.convert_to_tensor(filenames, dtype=tf.string)
    filename_q = tf.train.slice_input_producer([filename_tensor], num_epochs=None, shuffle=True)[0]

    image_255 = tf.image.decode_jpeg(tf.read_file(filename_q))
    image = tf.cast(image_255, tf.float32) / 255.0
    if grayscale:
        image = tf.image.grayscale_to_rgb(image)

    def resize_image(image):
        pre_crop_size = int(299 + random.random() * 100)
        image_as_batch_of_1 = tf.expand_dims(image, 0)
        image = tf.image.resize_bilinear(image_as_batch_of_1, [pre_crop_size, pre_crop_size])[0]
        image = tf.random_crop(image, [299, 299, 3])
        return image
    
    image = resize_image(image)

    def distort_image(image):
        # noise_amt = tf.abs(tf.random_normal([], stddev=0.2))
        # distorted_image = image + tf.random_uniform([299, 299, 3], maxval=noise_amt)
        distorted_image = tf.image.random_flip_left_right(image)
        distorted_image = tf.image.random_brightness(distorted_image, max_delta=0.3)
        distorted_image = tf.image.random_contrast(distorted_image, lower=0.6, upper=1.6)
        return distorted_image
    
    image = distort_image(image)
    
    images_batch, filenames_batch = tf.train.shuffle_batch([image, filename_q], batch_size=batch_size, capacity=128, min_after_dequeue=64)
    return images_batch, filenames_batch


In [4]:
class Classifier(object):
    def __init__(self, inception_path, n_classes, filenames, filename_to_class_fn):
        self.filenames = filenames
        self.filename_to_class_fn = filename_to_class_fn
        
        self.graph = tf.Graph()
                        
        with self.graph.as_default():
            graph_def, bottleneck, jpeg_input, resized_image_input, softmax_tensor = create_inception_graph(inception_path)
            
            with tf.variable_scope('additional'):
                identity = lambda x: x
                logits = tf.layers.dense(bottleneck, 256, tf.nn.relu, name='fc1')
                logits = tf.layers.dense(logits, n_classes, identity, name='fc2')
                labels = tf.placeholder(tf.int64, name='correct')

                loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)

                trainable = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='additional')
                step = tf.contrib.framework.get_or_create_global_step()
                train_step = tf.train.AdamOptimizer(1e-5).minimize(loss, var_list=trainable, global_step=step)

                predictions = tf.argmax(logits, axis=1)
                acc = tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

                self.logits = logits    
                self.resized_image_input = resized_image_input
                self.labels = labels
                self.loss = loss
                self.acc = acc
                self.train_step = train_step
                self.global_step = step
                self.predictions = predictions
                self.bottleneck = bottleneck

                self.images_batch, self.filenames_batch = create_image_queue(self.filenames, 1, grayscale=True)
                self.session = None
    
    def setup(self):
        with self.graph.as_default():
            self.session = tf.Session(graph=self.graph)
            # uninitialized = list(self.session.run(tf.report_uninitialized_variables()))
            # print uninitialized
            # uninitialized_vars = [tf.get_variable(v) for v in uninitialized]
            # self.session.run(tf.variables_initializer(uninitialized_vars))
            init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
            self.session.run(init_op)
            tf.train.start_queue_runners(sess=self.session)
    
    def train(self, batch_iterator):
        for x, y in batch_iterator:
            step = self.session.run(self.global_step)
            feed = {
                self.resized_image_input: x,
                self.labels: y
            }
            _, loss, acc, predictions = self.session.run([
                self.train_step, 
                self.loss, 
                self.acc, 
                self.predictions], feed_dict=feed)
            print "{}: {}, acc={}, pred={}, actual={}".format(step, loss, acc, predictions, y)
    
    def train_loop(self):
        if self.session is None:
            self.setup()
        
        def batch_gen():
            while True:
                images, filenames = self.session.run([self.images_batch, self.filenames_batch])
                x = images
                y = [self.filename_to_class_fn(filename) for filename in filenames]
                y = np.array(y, dtype=int)
                yield x, y
        
        self.train(batch_gen())


In [5]:
scenes_dir = '/Users/nateparrott/Documents/School/cv/proj4/data/train'
class_names = [c for c in os.listdir(scenes_dir) if not c.startswith('.')]
filename_to_class_map = {}
for class_idx, class_name in enumerate(class_names):
    class_path = os.path.join(scenes_dir, class_name)
    for name in os.listdir(class_path):
        if name.endswith('.jpg') or name.endswith('.png'):
            filename = os.path.join(class_path, name)
            filename_to_class_map[filename] = class_idx
n_classes = len(set(filename_to_class_map.values()))
print n_classes
filenames = filename_to_class_map.keys()
def filename_to_class(filename): return filename_to_class_map[filename]

15


In [6]:
x = Classifier(PATH, n_classes, filenames, filename_to_class)
print 'ok'

x.train_loop()

ok
0: [ 2.60544062], acc=0.0, pred=[4], actual=[13]
1: [ 3.74549961], acc=0.0, pred=[4], actual=[0]
2: [ 3.52336049], acc=0.0, pred=[4], actual=[0]
3: [ 2.57163], acc=0.0, pred=[4], actual=[12]
4: [ 2.39858818], acc=0.0, pred=[4], actual=[13]
5: [ 3.69894695], acc=0.0, pred=[4], actual=[11]
6: [ 3.25644779], acc=0.0, pred=[4], actual=[1]
7: [ 3.35061717], acc=0.0, pred=[4], actual=[3]
8: [ 3.74895191], acc=0.0, pred=[4], actual=[6]
9: [ 2.45582199], acc=0.0, pred=[4], actual=[10]
10: [ 4.14222336], acc=0.0, pred=[4], actual=[6]
11: [ 2.79571819], acc=0.0, pred=[4], actual=[2]
12: [ 3.55959988], acc=0.0, pred=[4], actual=[0]
13: [ 2.79764605], acc=0.0, pred=[4], actual=[14]
14: [ 2.0966692], acc=0.0, pred=[4], actual=[8]
15: [ 3.39124727], acc=0.0, pred=[4], actual=[3]
16: [ 3.66304851], acc=0.0, pred=[4], actual=[11]
17: [ 3.60063362], acc=0.0, pred=[4], actual=[0]
18: [ 2.75521612], acc=0.0, pred=[4], actual=[2]
19: [ 3.30293751], acc=0.0, pred=[4], actual=[3]
20: [ 3.91801119], acc=0

KeyboardInterrupt: 

In [ ]:
# show_graph(graph_def)


In [ ]:
%matplotlib inline

with tf.Session() as sess:
    images_batch, filenames_batch = create_image_queue(['/Users/nateparrott/Desktop/lucy.jpg'])

    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)
    tf.train.start_queue_runners(sess=sess)

    images, filenames = sess.run([images_batch, filenames_batch])
    print filenames[0]
    to_pil(images[0])

In [8]:

idx_to_class_id = {}
idx = None
for line in open(os.path.join(PATH, 'imagenet_2012_challenge_label_map_proto.pbtxt')):
    if 'target_class:' in line:
        idx = int(line.strip().split()[-1])
    elif 'target_class_string:' in line:
        class_id = line.split()[-1][1:-1]
        idx_to_class_id[idx] = class_id
        
class_id_to_name = {}
for line in open(os.path.join(PATH, 'imagenet_synset_to_human_label_map.txt')):
    if len(line.strip()):
        id, name = line.split(None, 1)
        class_id_to_name[id] = name

def idx_to_label(idx):
    return class_id_to_name[idx_to_class_id[idx]]

for idx in idx_to_class_id.keys():
    print idx_to_label(idx)

kit fox, Vulpes macrotis

English setter

Siberian husky

Australian terrier

English springer, English springer spaniel

grey whale, gray whale, devilfish, Eschrichtius gibbosus, Eschrichtius robustus

lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens

Egyptian cat

ibex, Capra ibex

Persian cat

cougar, puma, catamount, mountain lion, painter, panther, Felis concolor

gazelle

porcupine, hedgehog

sea lion

malamute, malemute, Alaskan malamute

badger

Great Dane

Walker hound, Walker foxhound

Welsh springer spaniel

whippet

Scottish deerhound, deerhound

killer whale, killer, orca, grampus, sea wolf, Orcinus orca

mink

African elephant, Loxodonta africana

Weimaraner

soft-coated wheaten terrier

Dandie Dinmont, Dandie Dinmont terrier

red wolf, maned wolf, Canis rufus, Canis niger

Old English sheepdog, bobtail

jaguar, panther, Panthera onca, Felis onca

otterhound, otter hound

bloodhound, sleuthhound

Airedale, Airedale terrier

hyena, hyaena

meerkat, mierk

In [ ]:
with tf.Session() as sess:
    test = '/Users/nateparrott/Desktop/lucy.jpg'
    jpeg_data = open(test).read()
    resp = sess.run(softmax_tensor, feed_dict={jpeg_input: jpeg_data})[0]
    top_indices = sorted(range(len(resp)), key=lambda idx: resp[idx], reverse=True)[:5]
    print map(idx_to_label, top_indices)
    